# 초음파 신호 처리
이번 포스트에서는 아날로그 초음파 진단기에서의 신호 흐름에 대해 설명할 예정이다.  

## 단일 소자 트랜스듀서를 사용한 B-모드 스캔
단일 소자 트랜스듀서를 통해 B-모드 스캔을 한다면 그림 1과 같은 방법으로 스캔이 진행된다.  
<p align="center"><img src="img/ultrasound16.png" width=500 height=250 />  

왼쪽의 방법은 리니어 스캔 방법은 초창기의 스캔방식으로 사용자의 숙련도에 따라 영상 차이가 많이 난다는 단점이 있다.  
오른쪽의 섹터 스캔 방법은 이후에 나온 스캔 방법으로 모터로 자동으로 구동이 되는 방식이다. 

---

위와 같이 스캔하여 얻은 신호들은 아래와 같은 방법으로 처리가 되어 영상이 구성된다.  

그림 2는 아날로그 초음파 진단기의 신호 흐름을 보여주는 그림이다.  
<p align="center"><img src="img/ultrasound15.jpg" width=750 height=350 />  

각 블록들에 대해 순차적으로 설명하자면 아래와 같다.  

### Transmitter (송신기)
Transmitter는 초음파를 발생시키기 위해 비교적 높은 펄스 전압을 발생하는 부분이다.

### Preamp (초단 증폭기)
Preamp는 되돌아온 초음파 에코 신호(*반사된 초음파 신호를 의미*)를 최초로 증폭하는 곳이다.  
여기서 유의해야할 점은 Preamp에서 증폭된 잡음 신호는 시스템 자체의 잡음 특성을 결정지을 수 있으므로 Preamp에서 잡음을 적게 발생시키도록 설계해야 한다는 점이다.

### TGC amp(Time-Gain-Compensation)or DGC amp(Depth-Gain-Compensation)
[초음파 물리 포스트](https://juwonk-bme.netlify.app/docs/bme/posts/ultrasound1#%EA%B0%90%EC%87%A0)에서 정리했듯이 초음파에서의 감쇠량은 아래와 같이 표현할 수 있다.  
$$감쇠량[dB] = 감쇠계수[dB/cm] \times 진행거리[cm] \times 주파수[MHz]$$  
즉, 진행거리 또는 깊이에 따라서 감쇠가 크게 되는데 이를 고려하여 영상을 구성하지 않으면 가까운 곳의 영상은 밝게 나오고 깊어질수록 어두운 영상을 얻게 된다.  
이러한 현상에 대한 대책이 **TGCamp** 혹은 **DGCamp**이다.  

> 초음파진단기에서 TGCamp와 DGCamp를 같게 보는 이유는 초음파가 깊은 곳에서 반사된다면 에코가 돌아오는 시간도 길어지기 때문이다.  

정리하면 TGCamp는 근거리의 echo신호는 거의 증폭하지 않고 먼 곳의 echo신호는 지수함수적으로 증폭시킨다.  
*지수함수적으로 증폭시키는 이유는 dB scale관점에서 보면 선형적이기 때문*  

아래 그림을 통해 적절한 TGC(*깊은 곳과 가까운 곳이 모두 밝게 보이는 것을 의미*)의 중요성을 볼 수 있다.  
<p align="center"><img src="img/ultrasound17.jpg" width=500 height=450 />  

근거리의 TGC가 낮다는 것은 근거리 echo 신호는 증폭하지 않았다는 것을 의미힌다. 그러므로 상대적으로 원거리 신호의 크기가 더 커지므로 근거리의 영상은 어둡게 표현되고 원거리 영상은 밝게 표현되게 된다. 이러한 TGC 조절을 TGC knobs을 통해 할 수 있다.

#### Receiver Gain (수신 이득)
전체적인 수신기의 증폭도를 줄이면 화면이 전체적으로 어두워져 보이고 키우면 화면이 전체적으로 밝아진다. 이러한 점만 본다면 송신 전압을 조절하는 것과 유사해보이지만 수신기의 증폭도는 노이즈 또한 조절이 된다는 점이 차이점이다. 예를 들면 수신기 증폭도를 올린다면 화면이 밝아짐과 동시에 노이즈도 커질 수 있다. 반면 수신기 증폭도는 송신 전압과 달라 초음파의 강도와는 상관이 없다.

### Envelope detection (포락선 검파)  

<p align="center"><img src="img/ultrasound18.png" />  

수신한 초음파 echo는 그림 4와 같이 +와 -를 오가는 펄스와 같은 모양을 갖는다. 이 신호를 바로 영상으로 만든다면 하나의 타겟이 여러 개로 오해될 수 있다.  
이러한 문제를 없애는 것, 즉 하나의 타겟을 영상에서 하나의 밝은 점으로 나타내기 위해 Envelope detection을 한다.  
그림 5는 간단한 Envelope detection의 예시이다. echo 신호를 절대값(전파 정류; 브릿지 회로를 통해 가능)취한 후 평활화(smoothing = low pass filtering)을 사용한 것이다.  

<p align="center"><img src="img/ultrasound19.png" />  

### Log Compression (대수 압축)  
**Dynamic range**: 최대 진폭과 최소 진폭의 비로 만약 최대 진폭이 10,000, 최소 진폭이 1이라면 Dynamic range는 아래와 같다.  
$$DR = 20\cdot \log_{10}(\frac{10000}{1})[dB]= 80[dB]$$  

위와 같이 DR이 $80[dB]$인 신호를 B모드 영상으로 사용한다면 밝은 점과 어두운 점의 밝기 차이가 매우 커서 영상장치에서 이를 표현하기가 어려워진다. 이를 해결하기 Log Compression을 사용한다.  

<p align="center"><img src="img/ultrasound20.png" />  

Log compression을 사용하면 그림 6과 같이 신호가 조정되어 어두운 부분은 비교적 밝게, 밝은 부분은 비교적 어둡게 표현된다.

#### Log compression 식 유도
Log compression의 입력을 $x$, 출력을 $y$라 하면 아래와 같은 형태로 표현할 수 있다.  
$$y = log comp(x) = A \cdot \log_{10}{x} + B$$  
아래와 같은 과정을 통해 A와 B를 유추할 수 있다.

`(1)`  
$y$의 범위는 $0 \le y < 1$이라 하고, $x$의 최대값을 1이라 가정하면  
$$y_{max} = A \cdot \log_{10}{1} + B = 1$$  
$$\therefore B = 1$$  

`(2)`  
$x$가 몇일 때 $y$가 0이 될지는 Dynamic range에 따라 결정된다.  
즉, $DR = 20 \cdot \log_{10}{\frac{x_{max}}{x_{min}}} = 20 \cdot \log_{10}{\frac{1}{x_{min}}}$이 된다.  
$x_{min} = 10^{-\frac{DR}{20}}$ 이므로 $y_{min}$은 아래와 같이 정리할 수 있다.  
$$y_{min} = logcomp(x_{min}) = A \cdot \log_{10}{10^{-\frac{DR}{20}}} + 1 =-A \cdot \frac{DR}{20} + 1 = 0$$
$$\therefore A = \frac{20}{DR}$$

$$
y = 
\begin{cases}
\frac{20}{DR} \cdot \log_{10}{x} + 1 \quad for \quad 10^{-\frac{DR}{20}} \le x \le 1\\
0 \quad \quad \quad \quad \quad \quad \quad for \quad x<10^{-\frac{DR}{20}}\\
\end{cases}
$$

일반적인 경우로 확장하면 다음과 같다.

$$
y = 
\begin{cases}
y_{max} \cdot \frac{20}{DR} \cdot \log_{10}{(\frac{x}{x_{max}})} + 1 \quad for \quad 10^{-\frac{DR}{20}} \le x \le 1\\
0 \quad \quad \quad \quad \quad \quad \quad \; \; \quad \quad \quad \quad for \quad \frac{x}{x_{max}}<10^{-\frac{DR}{20}}\\
\end{cases}
$$

위 수식에 적절한 값을 넣어 실험을 해보면 Log compression은 어두운 곳에서의 밝기 차이는 크게, 밝은 곳의 밝기 차이는 작게 표현해 줌을 알 수 있다.

#### Log compression에서 Dynamic range 조절에 따른 이미지 변화
앞에서 유도한 수식을 보면 알다시피 DR에 따라 신호의 크기가 많이 달라진다. 그림 7은 이를 표현한 그림이다.  
<p align="center"><img src="img/ultrasound21.png" />  

그림 7을 보면 DR이 크면 아주 높은 echo와 아주 낮은 echo를 모두 영상에 표현되는 것을 확인할 수 있다. 하지만 display에서 표현할 수 있는 정도는 일정하고 수식을 통해 알수 있듯 DR이 커지면 $y_{max}$는 작아지게 되어 echo간의 밝기 차이가 작아진다. 이로 인해 DR이 작은 영상은 명암비가 떨어지는 것처럼 보인다.  

반면 DR이 작으면 선명해 보이는 것처럼 보이지만 어느 이상 높은 echo 또는 어느 이상 낮은 echo는 가장 높은 밝기 또는 가장 낮은 밝기를 가져서 구분할 수 없게 된다.
